# Random Forest
In this document, we implement the random forest algorithm (machine learning model) from scratch in Python. To this end, we use the guidelines provided by AssemblyAI in https://www.youtube.com/watch?v=kFwe2ZZU7yw.

It is worth mentioning that, since random forest comprises multiple trees (decision trees), we import the DecisionTree class prvided in "decision_tree_scratch.ipynb".

## Import Libraries

In [1]:
import numpy as np
from collections import Counter
import import_ipynb
from decision_tree_scratch import DecisionTree as DecisionTree

## Class

In [2]:
class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=10, n_features=None,
                 mode='classification'):
        self.n_trees           = n_trees               # number of trees in the forest
        self.min_samples_split = min_samples_split     # stopping criterion: minimum number of samples in a leaf node
        self.max_depth         = max_depth             # stopping criterion: maximum depth of the tree
        self.n_features        = n_features            # number of features
        self.trees             = []                    # list of trees for the forest
        self.mode              = mode.lower()          # classification or regression

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(self.min_samples_split, self.max_depth, self.n_features, self.mode)
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X, y)
            self.trees.append(tree)

    def predict(self, X):
        preds_list  = np.array([tree.predict(X) for tree in self.trees])
        trees_preds  = np.swapaxes(preds_list, 0, 1)
        
        if self.mode == 'classification':
            # majority vote for classification
            predictions = np.array([self._most_common_label(preds) for preds in trees_preds])
        elif self.mode == 'regression':
            # average for regression
            predictions = np.mean(trees_preds, axis=1)
        
        return predictions

    ################################### Auxiliary Functions #####################################
    # form bootstrap (smapling) data
    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        
        return X[indices], y[indices]

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        
        return most_common

## Fit and Evaluate Model

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Classification

In [4]:
def classification():
    dataset = datasets.load_breast_cancer()
    X, y = dataset.data, dataset.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    clf = RandomForest()
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracy = np.sum(y_pred == y_test) / len(y_test)
    print(f"Results for Classification - Accuracy: {accuracy*100:.2f}")

### Regression

In [5]:
def regression():
    dataset = datasets.fetch_california_housing()
    X, y = dataset.data, dataset.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    reg = RandomForest(mode='regression')
    reg.fit(X_train, y_train)
    
    y_pred = reg.predict(X_test)
    loss = mean_squared_error(y_pred, y_test)
    print(f"Results for Regression - Loss: {loss:.4f}")

In [6]:
if __name__ == '__main__':
    classification()
    regression()

Results for Classification - Accuracy: 94.74
Results for Regression - Loss: 0.4143
